# Regression

## Table of Contents

1. [**Introduction**](#Intro)   
2. [**Non-Polynomial Regression**](#NonPoly)

    2.1 [**Model Development**](#MDevNP)

    2.2 [**Model Evaluation**](#MEvaNP)

3. [**Polynomial Regression**](#PolyDev)

    4.1 [**Model Development**](#MDevP)

    4.2 [**Model Evaluation**](#MEvaP)

4. [**Final Comments**](#FinalComments)


## 1. Introduction <a name="Intro"></a>

Linear models are governed by equations that weight each feature variable by a coefficient reflecting its importance, and sum up these values. But all phenomenon can be descibed using linear models. Richer mathematical descriptions might be needed, including higher-order polynomials, logarithms, and exponentials. These permit models to fit the training data much more tightly than linear functions.

That being said, more complicated models are not necessarily always better. Accoring to _Occam's razor_ __"the simplest explanation is the best explanation."__

## 2. Non-Polynomial Regression <a name="NonPoly"></a>

We can fit any function to our data in Python environment. To do so, we just need to define the function and then use `curve_fit` function from <font color='blue'>scipy</font> library, module `optimize`, to find the coefficients.

## 2.1 Model Development <a name="MDevNP"></a>

Here are the general steps you need to follow:

__1.__ Import curve fitting package from scipy library. Also, import train_test_split funcion from <font color='blue'>scikit_learn</font> library:
```python
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split
```

__2.__ Define dependant (target variable) and independent variable (feature) from data set:
```python
x_data=df['column1']
y_data=df['column2']
```

__3.__ Split the data into train and test sets: `x_train,x_test,y_train,y_test=train_test_split(x_data,y_data)`


__4.__ Define the function you want to fit to the data:
```python
def my_func(inputs):
  output=...
  return output
```

__5.__ Use the `curve_fit` function to fit the function to the training data and find the coefficients:
```python
popt, pcov = curve_fit(my_func, x_train, y_train)
```

__6.__ Then, make predictions using the test data

__7.__ Plot the training data, test data, and fitted curve to visually inspect the fit (optional)

Let's try this using a dataset representing strain rate sensitivity of cortical bone for longitudinal tensile loading. As with all biological materials, bone is viscoelastic and nonlinear. It has been shown that cortical bone is moderately strain-rate-dependent. This data is an excel sheet which includes stress-strain values for different strain rates.

sheet1 name='StrainRate_0.001' (0.001 percent strain per second)

sheet2 name='StrainRate_0.01'  (0.01 percent strain per second)

sheet3 name='StrainRate_0.1'   (0.1 percent strain per second)

sheet4 name='StrainRate_1'     (1 percent strain per second)

sheet5 name='StrainRate_300'   (300 percent strain per second)

sheet1 name='StrainRate_1500'  (1500 percent strain per second)

In [ ]:
import pandas as pd

xlsx = pd.ExcelFile('https://raw.githubusercontent.com/MasoudMiM/ME_364/master/StrainRate_CorticalBone/Cortical_Bone.xlsx')
df = pd.read_excel(xlsx,sheet_name='StrainRate_0.001', names=['Strain %','Stress (MPa)'])            # 0.001 percent strain per second

df.head()

First, let's take a look at the data we imported

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = '15'
df.plot(kind='scatter',x='Strain %',y='Stress (MPa)',figsize=(7,5))
plt.grid(True)

Now, let's define a function in the form of $y = a\frac{bx}{100+bx}$, where $a$ and $b$ are constants, and use it for our curve fitting procedure.

Steps __1__ and __2__, importing the libraries and defining the data for fitting

In [ ]:
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split

In [ ]:
x_data=df['Strain %']
y_data=df['Stress (MPa)']

Step __3__, splitting the data

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2,shuffle=True)

Step __4__, defining the function

In [ ]:
def my_func(x,a,b):
  output=a*(b*x/(100+b*x))
  return output

Step __5__, curve fitting and finding the coefficients

In [ ]:
popt, pcov = curve_fit(my_func, x_train, y_train)
#print the final parameters
print(f" a = {popt[0]:.3f}, b = {popt[1]:.3f}")

Step __6__, making predictions using test data

In [ ]:
yhat=my_func(x_test, popt[0], popt[1])

Step __7__, plotting the fitted curve along with training and test data

In [ ]:
import numpy as np

plt.rcParams['font.size'] = '15'
plt.figure(figsize=(8,7))

# plotting the training data
plt.scatter(x_train,y_train,color='blue',label='Training Data')

# Plotting the test data
plt.scatter(x_test,y_test,color='red',label='Test Data')

# plottign the fitted curve
xplot=np.linspace(0,max(x_train),num=50)
yplot=my_func(xplot,popt[0],popt[1])
plt.plot(xplot,yplot,color='green', lw=3,label='Fitted Curve')

# plots labeling and legend
plt.xlabel('Strain (%)')
plt.ylabel('Stress (MPa)')
plt.legend(loc='best');

## 2.2 Model Evaluation <a name="MEvaNP"></a>

To evalulate our model, we need to find and compare the errors for the test data. We are going to use Mean Squared Error (MSE) and R-Squared (R$^2$) for evaluating the performance of the model.


In [ ]:
# Mean Squared Error
from sklearn.metrics import mean_squared_error

MSETst=mean_squared_error(y_test,yhat)
print(f'The value of mean squared error is: {MSETst:0.3f}')

# R2
from sklearn.metrics import r2_score

r2scoreTst = r2_score(y_test,yhat)
print(f'The value of R2 is: {r2scoreTst:0.3f}')

Let's make a change in the function we used to fit the data and see how the performance changes. 

This time using the function Using $y=a\left(1-e^{-bx}\right)$, we get

In [ ]:
import numpy as np

def my_func2(x,a,b):
  output=a*(1-np.exp(-b*x))
  return output

popt2, pcov2 = curve_fit(my_func2, x_train, y_train)
#print the final parameters
print(f" a = {popt2[0]:0.3f}, b = {popt2[1]:0.3f}")

yhat2=my_func2(x_test, popt2[0], popt2[1])

plt.rcParams['font.size'] = '15'
plt.figure(figsize=(8,7))

# plotting the training data
plt.scatter(x_train,y_train,color='blue',label='Train Data')

# Plotting the test data
plt.scatter(x_test,y_test,color='red',label='Train Data')

# plottign the fitted curve
xplot2=np.linspace(0,1.75,num=50)
yplot2=my_func2(xplot,popt2[0],popt2[1])
plt.plot(xplot2,yplot2,color='green', lw=3,label='Fitted Curve')

# plots labeling and legend
plt.xlabel('Strain (%)')
plt.ylabel('Stress (MPa)')
plt.legend(loc='best');

Let's evaluate this model now

In [ ]:
# Mean Squared Error
MSE2=mean_squared_error(y_test, yhat2)
print(f'The value of mean squared error is: {MSE2:0.3f}')

# R2
r2score2 = r2_score(y_test,yhat2)
print(f'The value of R2 is: {r2score2:0.3f}')

## 3. Polynomial Regression <a name="PolyDev"></a>

Following our lecture, we convert the polynomial into a multiple linear regression. This process in Python can be done using `PloynomialFeatures` function in <font color='blue'>scikit-learn</font> library, which derives a new feature sets from the original feature set.

Once we transform the polynomial, we can follow the same steps that we did for multiple linear regression (previous class) to develop a regression model.

## 3.1 Model Development <a name="MDevP"></a>

Here are the steps to implement polynomial regression in Python using scikit-learn.

__1.__ Import Numpy, PolynomialFeatures, LinearRegression and train_test_split funcions from <font color='blue'>scikit_learn</font> library:
```python
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
```

__2.__ Define dependant variable (target variable) and independent variable (feature) from data set:
```python
x_data=np.array(df[['column1']])
y_data=np.array(df[['column2']])
```

__3.__ Split the data into train and test sets: `x_train,x_test,y_train,y_test=train_test_split(x_data,y_data)`

__4.__ Transform the training and test data using PolynomialFeatures() function and knowing the degree of polynomial you want.
```python
poly = PolynomialFeatures(degree=2)  # for a 2 degrre polynomial
x_train_transformed = poly.fit_transform(x_train)
x_test_transformed = poly.fit_transform(x_test)
```

__5.__ Create a linear regression object using the constructor: `lmPoly = LinearRegression() `


__6.__ Use the fit function to fit the model to the training data and find the coefficients: `lmPoly.fit(x_train_transformed, y_train)`

__7.__ Then, make prediction using the test data and training data:
```python
yhatTest =lmPoly.predict(x_test_transformed)
yhatTrain=lmPoly.predict(x_train_transformed)
```

__8.__ Plot the training data, test data, and fitted curve to visually inspect the fit (optional)

Before going further, I recomend cleaning the memory and clear all the variables defined in this notebook so far...

In [ ]:
%reset

Let's bring our 3D printer data set and use that for model development.

Parameters:

- Layer Height (mm)

- Wall Thickness (mm)

- Infill Density (%)

- Infill Pattern ()

- Nozzle Temperature (Cº)

- Bed Temperature (Cº)

- Print Speed (mm/s)

- Material ()

- Fan Speed (%)

- Roughness (µm)

- Tnesile (ultimate) Strenght (MPa)

- Elongation (%) 


In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/MasoudMiM/ME_364/main/3D_Printer_Data/3DPrinterDataset.csv'   # Link to the 3D printer data set
df = pd.read_csv(url)

df.head()

Just as a quick look at the data, I am going to use `pairplot` function from <fonr color='blue'> seaborn </font> library. (this might take a few minutes to run)

In [ ]:
import seaborn as sns

sns.pairplot(df)

Let us take a look at roughness and layer height. We try to train a model to predict roughness based on the layer height.

Steps __1__ and __2__, importing required modules and functions as well as defining the variables.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
x_dataPoly=np.array(df[['layer_height']])
y_dataPoly=np.array(df[['roughness']])

Step __3__, splitting the data into test and train data

In [ ]:
x_trainP,x_testP,y_trainP,y_testP=train_test_split(x_dataPoly, y_dataPoly, test_size=0.2, random_state=125)

Step __4__, transforming the data

In [ ]:
poly = PolynomialFeatures(degree=2)  # for a 2nd degree polynomial
x_train_transformed = poly.fit_transform(x_trainP)
x_test_transformed = poly.fit_transform(x_testP)

Step __5__, create a linear regression instance

In [ ]:
lmPoly = LinearRegression()

Step __6__, use the fit function to fit the model to the training data and find the coefficients

In [ ]:
lmPoly.fit(x_train_transformed, y_trainP)

# printing the coefficients
print('w_0=',lmPoly.intercept_[0],',w_1=',lmPoly.coef_[0][1], ', and w_2=',lmPoly.coef_[0][2])

Step __7__, make predictions

In [ ]:
yhatPt=lmPoly.predict(x_test_transformed)   # using testing data
yhatPTr=lmPoly.predict(x_train_transformed) # using training data

Step __8__, plotting the fitted curve, training data, and test data.

In [ ]:
import matplotlib.pyplot as plt

# figure size and font size
plt.figure(figsize=(10,8))
plt.rc('font',size=16)

# Plotting the fitted curve
xplotP=np.linspace(0.025,0.2,num=40)
yplotP=lmPoly.intercept_[0]+lmPoly.coef_[0][1]*xplotP+lmPoly.coef_[0][2]*xplotP**2
plt.plot(xplotP,yplotP,color='green', lw=3, label='Fitted Curve')

# Plotting the training data
plt.scatter(x_trainP,y_trainP,color='blue',s=80, label='Training Data')

# Plotting the test data
plt.scatter(x_testP,y_testP,color='red',s=80, label='Test Data')

# Plot labels and legend
plt.xlabel('Elongation (%)')
plt.ylabel('Roughness (micro-meter)')
plt.legend(loc='best');

Let's now do the same process and use a higher order polynomial

In [ ]:
# Defining the polynomial and fit it into the data
poly4 = PolynomialFeatures(degree=4)  # for a 4th degree polynomial
x_train_transformedP4 = poly4.fit_transform(x_trainP)
x_test_transformedP4 = poly4.fit_transform(x_testP)
lmPoly4 = LinearRegression()
lmPoly4.fit(x_train_transformedP4, y_trainP)

# Predictions
yhatP4t=lmPoly4.predict(x_test_transformedP4)
yhatP4Tr=lmPoly4.predict(x_train_transformedP4)

# figure size and font size
plt.figure(figsize=(10,8))
plt.rc('font',size=16)

# Plotting the fitted curve
yplotP4=lmPoly4.intercept_[0]+lmPoly4.coef_[0][1]*xplotP+lmPoly4.coef_[0][2]*xplotP**2+lmPoly4.coef_[0][3]*xplotP**3+lmPoly4.coef_[0][4]*xplotP**4
plt.plot(xplotP,yplotP4,color='green', lw=3, label='Fitted Curve')

# Plotting the training data
plt.scatter(x_trainP,y_trainP,color='blue',s=80, label='Training Data')

# Plotting the test data
plt.scatter(x_testP,y_testP,color='red',s=80, label='Test Data')

# Plot labels and legend
plt.xlabel('Elongation (%)')
plt.ylabel('Roughness (micro-meter)')
plt.legend(loc='best');


## 3.2 Model Evaluation <a name="MEvaP"></a>

Let us evaluate our models using relative squared error (RSE). I start with the 2$^{nd}$ degree polynomial regression. 

In [ ]:
from sklearn.metrics import r2_score

## Evaluation using test data
rse2ndp_test = 1-r2_score(y_testP, yhatPt)
print('The value of RSE for test data for 2nd degree polynomial regression is: %.2f\n' %rse2ndp_test)

# Evaluation using train data
rse2ndp_train = 1-r2_score(y_trainP, yhatPTr)
print('The value of RSE for training data for 2nd degree polynomial regression is: %.2f' %rse2ndp_train)

Now, let's take a look at RSE for 4$^{th}$ degree polynomial regression

In [ ]:
## Evaluation using test data
rse4thp_test = 1-r2_score(y_testP,yhatP4t)
print('The value of RSE for test data for 4th degree polynomial regression is: %.2f' %rse4thp_test)

print('')
# Evaluation using train data
rse4thp_train = 1-r2_score(y_trainP,yhatP4Tr)
print('The value of RSE for training data for 4th degree polynomial regression is: %.2f' %rse4thp_train)

Let's try to do the regression process for polynomials with different degrees, find the relative squared errors of training and test data for each.

In [ ]:
degreep=[0,1,2,3,4,5,6]
rsePolyNom_test=np.zeros((len(degreep),1))
rsePolyNom_train=np.zeros((len(degreep),1))

for ii,deg in enumerate(degreep):
  # Defining the polynomial and fit it into the data
  polyNom = PolynomialFeatures(degree=deg) 
  x_train_transformedPolyNm = polyNom.fit_transform(x_trainP)
  x_test_transformedPolyNm = polyNom.fit_transform(x_testP)
  lmPolyNom = LinearRegression()
  lmPolyNom.fit( x_train_transformedPolyNm , y_trainP)
  
  # calculating rse for test data
  yhatPolyNomTS=lmPolyNom.predict(x_test_transformedPolyNm)
  rsePolyNom_test[ii]=1-r2_score(y_testP,yhatPolyNomTS)

  # calculating rse for training data
  yhatPolyNomTR=lmPolyNom.predict(x_train_transformedPolyNm)
  rsePolyNom_train[ii]=1-r2_score(y_trainP,yhatPolyNomTR)

print('Test Errors:')
print(rsePolyNom_test)
print('')
print('Train Errors:')
print(rsePolyNom_train)

plt.figure(figsize=(6,6))
plt.rc('font',size=12)
plt.plot(degreep,rsePolyNom_test,'-o',degreep,rsePolyNom_train,'-o', linewidth=2,markersize=10)
plt.xlabel('Polynomial Degree')
plt.ylabel('Relative Squared Error')
plt.legend(['Test Data','Training Data']);

Let's plot these polynomials and see what do they look like

In [ ]:
degreep=[1,2,3,4,5,6]

# setting figure size and font size
fig=plt.figure(figsize=(15,8))
plt.rc('font',size=8)

# For plotting the fitted curve
xplotP=np.linspace(0.025,0.2,num=40)

for ii, deg in enumerate(degreep):
  fig.add_subplot(2,3,ii+1)
  
  # Defining the polynomial and fit it into the data
  polyNomp = PolynomialFeatures(degree=deg) 
  x_train_transformedPolyNmp = polyNomp.fit_transform(x_trainP)
  x_test_transformedPolyNmp = polyNomp.fit_transform(x_testP)
  lmPolyNomp = LinearRegression()
  lmPolyNomp.fit( x_train_transformedPolyNmp , y_trainP )

  yplottmp=np.zeros((xplotP.shape))
  yplotP=np.zeros((xplotP.shape))

  for jj in range(deg):
    yplottmp=yplottmp+lmPolyNomp.coef_[0][jj+1]*xplotP**(jj+1)

  print('')
  yplotP=lmPolyNomp.intercept_[0]+yplottmp
  plt.plot(xplotP,yplotP,color='green', lw=3, label='Fitted Curve')
  
  # Plotting the training data
  plt.scatter(x_trainP,y_trainP,color='blue',s=80, label='Training Data')
  
  # Plotting the test data
  plt.scatter(x_testP,y_testP,color='red',s=80, label='Test Data')
  
  # Plot labels and legend
  plt.title('{}-order polynomial'.format(deg))
  plt.xlabel('Elongation (%)')
  plt.ylabel('Roughness (micro-meter)')
  plt.legend(loc='best')

  # set the spacing between subplots
plt.subplots_adjust(top=0.8, wspace=0.4, hspace=0.4)

## 4. Final Comments <a name="FinalComments"></a>

When we are developing a regression model, we need to look at the errors for both training data and test data. Keep in mind that simple models can underfit the data while more complicated models can overfit the data. When the model underfits the data, we have high bias and when the model overfits the data, we have high variance. 
